## Deliverable 3. Create a Travel Itinerary Map.

In [45]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [46]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,4,Atuona,PF,77.23,few clouds,-9.8000,-139.0333,Villa Enata
1,11,Komsomolskiy,UZ,77.72,clear sky,40.4272,71.7189,Ikathouse
2,15,Vaini,TO,77.16,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
3,17,Yamada,JP,77.99,overcast clouds,39.4667,141.9500,Iroha-ya Inn
4,21,Soe,ID,89.51,broken clouds,-9.8607,124.2840,Dena Hotel


In [47]:
#Determine if there are any empty rows.
vacation_df.isnull().sum()

City_ID                 0
City                    0
Country                 0
Max Temp                0
Current Description     0
Lat                     0
Lng                     0
Hotel Name             12
dtype: int64

In [48]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_hotel_df = vacation_df.dropna()
clean_hotel_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,4,Atuona,PF,77.23,few clouds,-9.8000,-139.0333,Villa Enata
1,11,Komsomolskiy,UZ,77.72,clear sky,40.4272,71.7189,Ikathouse
2,15,Vaini,TO,77.16,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
3,17,Yamada,JP,77.99,overcast clouds,39.4667,141.9500,Iroha-ya Inn
4,21,Soe,ID,89.51,broken clouds,-9.8607,124.2840,Dena Hotel
...,...,...,...,...,...,...,...,...
192,654,Denpasar,ID,82.06,scattered clouds,-8.6500,115.2167,OYO 3156 Hotel Warta 2
193,655,Nortelandia,BR,80.94,broken clouds,-14.4547,-56.8028,Pousada e Restaurante Santana
194,657,Gold Coast,AU,85.77,clear sky,-28.0000,153.4333,Mantra Towers of Chevron Surfers Paradise
195,659,Woodward,US,79.83,clear sky,36.4337,-99.3904,"Holiday Inn Express & Suites Woodward Hwy 270,..."


In [49]:
#Determine if there are any empty rows.
clean_hotel_df.isnull().sum()

City_ID                0
City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [50]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dd>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [7]:
# 4a. Add a marker layer for each city to the map.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [51]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = clean_hotel_df.loc[clean_hotel_df["City"] == "Zlitan"]
vacation_end = clean_hotel_df.loc[clean_hotel_df["City"] == "Zlitan"]
vacation_stop1 = clean_hotel_df.loc[clean_hotel_df["City"] =="Hun"]
vacation_stop2 = clean_hotel_df.loc[clean_hotel_df["City"] =="Benghazi"]
vacation_stop3 = clean_hotel_df.loc[clean_hotel_df["City"] == "Tukrah"]
vacation_start

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
58,232,Zlitan,LY,75.25,clear sky,32.4674,14.5687,القسم الداخلي للجامعة الأسمرية


In [52]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start['Lat'].to_numpy()[0], vacation_start['Lng'].to_numpy()[0]
end = vacation_end['Lat'].to_numpy()[0], vacation_end['Lng'].to_numpy()[0]
stop1 = vacation_stop1['Lat'].to_numpy()[0], vacation_stop1['Lng'].to_numpy()[0]
stop2 = vacation_stop2['Lat'].to_numpy()[0], vacation_stop2['Lng'].to_numpy()[0]
stop3 = vacation_stop3['Lat'].to_numpy()[0], vacation_stop3['Lng'].to_numpy()[0]


In [24]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
vacation_itinerary = gmaps.directions_layer(
        start, end, waypoints=[stop1, stop2, stop3],
        travel_mode='DRIVING')

fig.add_layer(vacation_itinerary)
fig


Figure(layout=FigureLayout(height='420px'))

In [26]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,232,Zlitan,LY,75.25,clear sky,32.4674,14.5687,القسم الداخلي للجامعة الأسمرية
1,355,Hun,LY,78.67,clear sky,29.1268,15.9477,Alrwasi Hotel
2,155,Benghazi,LY,77.77,clear sky,32.1167,20.0667,Tibesti Hotel
3,447,Tukrah,LY,76.32,few clouds,32.5341,20.5791,Lagrota restaurant


In [43]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dd>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [44]:
# 11a. Add a marker layer for each city to the map.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))